# Aegis Memory Operations

Aegis Memory provides 12 RL-trained operations for AI agent memory management.
This notebook demonstrates all 12 operations with a legal domain example.

**Operations:** store, retrieve, update, forget, search, link, promote, demote,
compress, verify, annotate, merge

In [ ]:
from datetime import UTC, datetime

from aegis import AegisMemory
from aegis.core.types import MemoryTier, TemporalBounds

## Setup

Create an in-memory AegisMemory instance. No external services needed.

In [ ]:
memory = AegisMemory(
    domain="legal",
    customer_id="sterling-associates",
    agent_id="legal-analyst-v1",
    enable_vectors=False,  # Fast startup, uses fuzzy matching fallback
)
manager = memory._manager  # For operations 8-12
print(f"Memory instance: {memory}")

## 1. Store

Store entries with full provenance, confidence, temporal bounds, and tags.

In [ ]:
# Store contract parties with provenance
entry = memory.store(
    key="contract:parties",
    value="Sterling & Associates LLP and Meridian Consulting Group Inc.",
    tier=MemoryTier.SESSION,
    confidence=0.95,
    provenance={
        "source_type": "document",
        "source_ref": "services-agreement-2026-01.pdf",
        "page": 1,
    },
    temporal_bounds=TemporalBounds(
        valid_from=datetime(2026, 2, 1, tzinfo=UTC),
        valid_to=datetime(2026, 7, 31, tzinfo=UTC),
    ),
    tags=["contract", "parties", "legal"],
)
print(f"Stored: key={entry.key}, tier={entry.tier.value}, confidence={entry.confidence}")

# Store additional entries
memory.store(key="contract:effective_date", value="January 15, 2026",
             tier=MemoryTier.SESSION, confidence=0.99, tags=["contract", "date"])
memory.store(key="contract:payment_terms",
             value="Monthly retainer of $45,000 USD, payable within 30 days.",
             tier=MemoryTier.SESSION, confidence=0.97, tags=["contract", "payment"])
memory.store(key="contract:confidentiality",
             value="3-year confidentiality obligation following termination.",
             tier=MemoryTier.SESSION, confidence=0.98, tags=["contract", "confidentiality"])
memory.store(key="temp:draft_note", value="Need to review indemnification clause.",
             tier=MemoryTier.WORKING, confidence=0.5, tags=["draft"])
print("Stored 5 entries total.")

## 2. Retrieve

In [ ]:
parties = memory.retrieve("contract:parties")
print(f"Retrieved: {parties}")

# Dict-like access
missing = memory.get("nonexistent:key", default="<not found>")
print(f"Get with default: {missing}")
print(f"'contract:parties' in memory: {'contract:parties' in memory}")

## 3. Update

In [ ]:
updated = memory.update(
    "contract:payment_terms",
    "Monthly retainer of $45,000 USD, payable within 30 days. "
    "Late payments accrue 1.5% monthly interest.",
)
print(f"Updated: {updated}")
print(f"New value: {memory.retrieve('contract:payment_terms')}")

## 4. Forget

In [ ]:
forgotten = memory.forget("temp:draft_note")
print(f"Forgot 'temp:draft_note': {forgotten}")
print(f"After forget: {memory.retrieve('temp:draft_note')}")

## 5. Search

Semantic search across all stored entries.

In [ ]:
results = memory.search("payment obligations and financial terms", top_k=5)
print("Search results:")
for r in results:
    print(f"  key={r['key']}, confidence={r['confidence']:.2f}, value={str(r['value'])[:60]}...")

## 6. Link

Create relationships between memory entries.

In [ ]:
linked = memory.link("contract:parties", "contract:payment_terms", relation="has_obligation")
print(f"Linked parties -> payment: {linked}")

linked2 = memory.link("contract:parties", "contract:confidentiality", relation="subject_to")
print(f"Linked parties -> confidentiality: {linked2}")

## 7. Promote

Move entries to higher-priority memory tiers.

In [ ]:
promoted = memory.promote("contract:parties", target_tier=MemoryTier.PERMANENT)
print(f"Promoted to PERMANENT: {promoted}")

entry = manager.retrieve("contract:parties")
if entry:
    print(f"Verified tier: {entry.tier.value}")

## 8. Demote (via MemoryManager)

In [ ]:
demoted = manager.demote(
    "contract:effective_date",
    target_tier=MemoryTier.WORKING,
    agent_id="legal-analyst-v1",
    customer_id="sterling-associates",
)
print(f"Demoted to WORKING: {demoted}")

## 9. Compress

In [ ]:
compressed = manager.compress(
    "contract:payment_terms",
    agent_id="legal-analyst-v1",
    customer_id="sterling-associates",
)
print(f"Compressed: {compressed}")
entry = manager.retrieve("contract:payment_terms")
if entry:
    print(f"Value: {entry.value[:80]}...")

## 10. Verify

In [ ]:
verified = manager.verify(
    "contract:parties",
    agent_id="legal-analyst-v1",
    customer_id="sterling-associates",
)
print(f"Verified: {verified}")

## 11. Annotate

In [ ]:
annotated = manager.annotate(
    "contract:confidentiality",
    annotation={
        "reviewer": "senior-counsel",
        "risk_level": "low",
        "notes": "Standard 3-year NDA clause.",
    },
    agent_id="legal-analyst-v1",
    customer_id="sterling-associates",
)
print(f"Annotated: {annotated}")

## 12. Merge

Combine two related entries into one.

In [ ]:
manager.store(key="clause:termination_a",
              value="Either party may terminate with 30 days written notice.",
              tier=MemoryTier.SESSION, agent_id="legal-analyst-v1",
              customer_id="sterling-associates")
manager.store(key="clause:termination_b",
              value="Immediate termination for material breach after 10-day cure period.",
              tier=MemoryTier.SESSION, agent_id="legal-analyst-v1",
              customer_id="sterling-associates")

merged = manager.merge(
    "clause:termination_a", "clause:termination_b",
    agent_id="legal-analyst-v1",
    customer_id="sterling-associates",
)
if merged:
    print(f"Merged key:   {merged.key}")
    print(f"Merged value: {merged.value}")
    print(f"Merged tier:  {merged.tier.value}")

## Health Dashboard & Audit Trail

In [ ]:
health = memory.health()
print(f"Total entries:  {health['total_entries']}")
print(f"Tier counts:    {health['tier_counts']}")
print(f"Event log size: {health['event_log_size']}")
print(f"Integrity OK:   {health['integrity_ok']}")

print("\nAudit trail (last 5):")
for event in memory.audit_trail()[-5:]:
    print(f"  [{event['timestamp'][:19]}] {event['operation']:>10s}  key={event['key']}")

## Next Steps

- Use `enable_vectors=True` for real semantic search (requires `sentence-transformers`)
- Connect to PostgreSQL/pgvector for persistent storage
- Connect to Neo4j for knowledge graph links
- See `04_ingestion_pipeline.ipynb` for feeding documents into memory